In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

**LSTM**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class LSTMModel(nn.Module):
    def __init__(self, input_size):
        super(LSTMModel, self).__init__()
        self.gru1 = nn.LSTM(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)
        return x

# Function to initialize model weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

# Input shape
input_shape = (80, 636228, 1)

# Create the PyTorch model with weight initialization
model = GRUModel(input_shape[1])
model.apply(init_weights)
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training, Validation, and Testing loops
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_loss_train = 0
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)
        batch_y = batch_y.float()

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    # Calculate average training loss for the epoch
    average_loss_train = total_loss_train / len(train_loader)
    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)
            batch_y_val = batch_y_val.float()

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    # Print metrics for the epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss_train}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')

    # Adjust learning rate using scheduler
    scheduler.step()


GRUModel(
  (gru1): LSTM(636228, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Average Loss: 0.7154402919113636, Training Accuracy: 51.25%, Validation Accuracy: 36.36%
Epoch [2/10], Average Loss: 0.7033217065036297, Training Accuracy: 47.50%, Validation Accuracy: 45.45%
Epoch [3/10], Average Loss: 0.696284182369709, Training Accuracy: 48.75%, Validation Accuracy: 45.45%
Epoch [4/10], Average Loss: 0.6923133693635464, Training Accuracy: 50.00%, Validation Accuracy: 45.45%
Epoch [5/10], Average Loss: 0.689653780311346, Training Accuracy: 50.00%, Validation Accuracy: 45.45%
Epoch [6/10], Average Loss: 0.6891142316162586, Training Accuracy: 50.00%, Validation Accuracy: 45.45%
Epoch [7/10], Average Loss: 0.6927569024264812, Training Accuracy: 50.00%, Validation Accuracy: 45.45%
Epoch [8/10], Average Loss: 0.6898260712623596, Training Accuracy: 50.00%, Validation Accuracy: 40.91%
Epoch [9/10], Average Loss: 0.69148332

**GRU**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])  # If the input has three dimensions, get the last output
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)  # If the input has two dimensions, add and squeeze a dimension
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)  # Adjust to have the same size as the target
        return x







input_shape = (80, 636228, 1)


model = GRUModel(input_shape[1])
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)  # Change dtype to float32 for binary classification
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)  # Change dtype to float32 for binary classification
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False) 

criterion = nn.BCEWithLogitsLoss()  # Change loss function for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training, Validation, and Testing loops
num_epochs = 10


for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)

        # Ensure the target tensor is in the range [0, 1]
        batch_y = batch_y.float()  # Convert to float
        #batch_y = batch_y.unsqueeze(1)  # Ensure it has the correct shape

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)

            # Ensure the target tensor is in the range [0, 1]
            batch_y_val = batch_y_val.float()  # Convert to float
            batch_y_val = batch_y_val.unsqueeze(1)  # Ensure it has the correct shape

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')






GRUModel(
  (gru1): GRU(636228, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Loss: 0.7357641458511353, Training Accuracy: 45.00%, Validation Accuracy: 222.73%
Epoch [2/10], Loss: 0.8199621438980103, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [3/10], Loss: 0.6849316954612732, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [4/10], Loss: 0.7496523261070251, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [5/10], Loss: 0.6766282320022583, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [6/10], Loss: 0.6851674318313599, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [7/10], Loss: 0.6269125938415527, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [8/10], Loss: 0.7277967929840088, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [9/10], Loss: 0.7023771405220032, Training Accuracy: 50.00%, Validation Accuracy: 222.7

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)
        return x

# Function to initialize model weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

# Input shape
input_shape = (80, 636228, 1)

# Create the PyTorch model with weight initialization
model = GRUModel(input_shape[1])
model.apply(init_weights)
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training, Validation, and Testing loops
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_loss_train = 0
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)
        batch_y = batch_y.float()

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    # Calculate average training loss for the epoch
    average_loss_train = total_loss_train / len(train_loader)
    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)
            batch_y_val = batch_y_val.float()

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    # Print metrics for the epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss_train}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')

    # Adjust learning rate using scheduler
    scheduler.step()


GRUModel(
  (gru1): GRU(636228, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Average Loss: 0.7094924040138721, Training Accuracy: 48.75%, Validation Accuracy: 45.45%
Epoch [2/10], Average Loss: 0.7028701528906822, Training Accuracy: 48.75%, Validation Accuracy: 50.00%
Epoch [3/10], Average Loss: 0.6911830827593803, Training Accuracy: 51.25%, Validation Accuracy: 54.55%
Epoch [4/10], Average Loss: 0.6893772594630718, Training Accuracy: 52.50%, Validation Accuracy: 54.55%
Epoch [5/10], Average Loss: 0.70161347463727, Training Accuracy: 48.75%, Validation Accuracy: 54.55%
Epoch [6/10], Average Loss: 0.6885979510843754, Training Accuracy: 50.00%, Validation Accuracy: 54.55%
Epoch [7/10], Average Loss: 0.6973218694329262, Training Accuracy: 50.00%, Validation Accuracy: 54.55%
Epoch [8/10], Average Loss: 0.6945449151098728, Training Accuracy: 51.25%, Validation Accuracy: 54.55%
Epoch [9/10], Average Loss: 0.690007198

VGG16

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
from skimage.transform import resize

class ModifiedVGG16(tf.keras.Model):
    def __init__(self, num_classes=2, input_channels=10):
        super(ModifiedVGG16, self).__init__()
        self.features = models.Sequential([
            layers.Conv2D(64, kernel_size=3, padding='same', activation='relu', input_shape=(224, 224, input_channels)),
            layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2, strides=2),

            layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2, strides=2),

            layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2, strides=2),

            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2, strides=2),

            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2, strides=2)
        ])
        self.avgpool = layers.GlobalAveragePooling2D()
        self.classifier = models.Sequential([
            layers.Dense(4096, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4096, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes)
        ])

    def call(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

# Input shape
input_shape = (20, 80, 636228)  # Assuming you have spectrogram data with 20 channels

# Resize the spectrogram to a size compatible with VGG16 (e.g., 224x224)
def custom_resize(data):
    resized_data = np.zeros((data.shape[0], 224, 224, data.shape[3]))
    for i in range(data.shape[0]):
        for j in range(data.shape[3]):
            resized_data[i, :, :, j] = resize(data[i, :, :, j], (224, 224))
    return resized_data

# Create the TensorFlow model
model = ModifiedVGG16(num_classes=2, input_channels=10)
model.build((None, 224, 224, 10))  # Build the model with a specific input shape
print(model.summary())

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

# Reshape and resize the input data
X_train_tensor = custom_resize(TrainF)
y_train_tensor = TrainL
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor)).shuffle(10000).batch(10)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = custom_resize(ValF)
y_val_tensor = ValL
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_tensor, y_val_tensor)).batch(5)

# Define the criterion and optimizer
criterion = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=0.001)

# Training and Validation loop
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_dataset:
        outputs = model(batch_X)
        loss = criterion(batch_y, outputs)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        predicted_train = tf.argmax(outputs, axis=1)
        total_correct_train += tf.reduce_sum(tf.cast(tf.equal(predicted_train, batch_y), dtype=tf.float32)).numpy()
        total_samples_train += batch_y.shape[0]

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    total_correct_val = 0
    total_samples_val = 0

    for batch_X_val, batch_y_val in val_dataset:
        outputs_val = model(batch_X_val)
        predicted_val = tf.argmax(outputs_val, axis=1)
        total_correct_val += tf.reduce_sum(tf.cast(tf.equal(predicted_val, batch_y_val), dtype=tf.float32)).numpy()
        total_samples_val += batch_y_val.shape[0]

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.numpy()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')






Model: "modified_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 7, 7, 512)         14718720  
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                                         
                                                                 
 sequential_1 (Sequential)   (None, 2)                 18890754  
                                                                 
Total params: 33609474 (128.21 MB)
Trainable params: 33609474 (128.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


IndexError: tuple index out of range

**SVM**

In [26]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = SVC(kernel='rbf', C=4)


svm_model.fit(TrainF, TrainL)

y_pred = svm_model.predict(ValF)


accuracy = accuracy_score(ValL, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 68.18%


**Naive Bayes**

In [27]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb_model = GaussianNB()

nb_model.fit(TrainF, TrainL)

y_pred = nb_model.predict(ValF)

# Evaluate the accuracy of the model
accuracy = accuracy_score(ValL, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 50.00%


**Random Forest**

In [28]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you already have TrainF, TrainL, ValF, and ValL

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the model on the training data
rf_model.fit(TrainF, TrainL)

# Make predictions on the test data
y_pred_rf = rf_model.predict(ValF)

# Evaluate the accuracy of the Random Forest model
accuracy_rf = accuracy_score(ValL, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")

Random Forest Accuracy: 54.55%
